<a href="https://colab.research.google.com/github/Iamkillua/OneClickRun/blob/master/OneClickRun.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#`mountUmount(`<font size="3px" color="#01c968">`Gdrive`</font>`)`



In [ ]:
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/Google_Drive_logo.png/600px-Google_Drive_logo.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Mount Gdrive to /content/drive</h3></center><br>
MODE = "MOUNT" #@param ["MOUNT", "UNMOUNT"]
#Mount your Gdrive! 
from google.colab import drive
drive.mount._DEBUG = False
if MODE == "MOUNT":
  drive.mount('/content/drive', force_remount=True)
elif MODE == "UNMOUNT":
  try:
    drive.flush_and_unmount()
  except ValueError:
    pass
  get_ipython().system_raw("rm -rf /root/.config/Google/DriveFS")

#`useFulTools(`<font size="3px" color="#a05fc3">`Jellyfin` </font> , <font size="3px" color="#ff55cc">`V2Ray` </font> , <font size="3px" color="#ffce42">`pyLoad` </font> , <font size="3px" color="#2ebd59">`spotify-downloader` </font> , <font size="3px" color="#21b5f7">`aria2Rpc` </font> , <font size="3px" color="#85ffbb"> `aria2Cli`</font> , <font size="3px" color="#88caf0"> `zip-tar-rar`</font>`)`



## <img src='https://github.com/pyload/pyload-webui/raw/master/media/banner.png' height="60" alt="pyLoad" />

In [ ]:
#@markdown <h3>⬅️ Click Here to START server</h3>
#@markdown <br><center><img src='https://github.com/pyload/pyload-webui/raw/master/media/banner.png' height="100" alt="pyLoad"/></center>
#@markdown <center><h3>pyLoad Webinterface<br />The free and open-source Download Manager written in pure Python</h3></center><br>
import os, pathlib, zipfile, re, threading
import urllib.request
from IPython.display import HTML, clear_output

Version = '1.3.0'

#####################################
USE_FREE_TOKEN = True  # @param {type:"boolean"}
TOKEN = ""  # @param {type:"string"}
REGION = "SA" #@param ["US", "EU", "AP", "AU", "SA", "JP", "IN"]
PORT_FORWARD = "argotunnel" #@param ["ngrok", "localhost", "argotunnel"]
#@markdown Default <br>&emsp;username : admin<br>&emsp;password : admin
HOME = os.path.expanduser("~")

if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")


from ocr import (
    runSh,
    loadingAn,
    PortForward_wrapper,
    findProcess
)

loadingAn()

os.makedirs('downloads', exist_ok=True)
os.makedirs('tools/pyload', exist_ok=True)

# Downloading latest version of pyload
if not os.path.exists("tools/pyload/pyload-stable"):
  urlF = 'https://github.com/pyload/pyload/archive/stable.zip'
  conf = 'https://raw.githubusercontent.com/biplobsd/' \
          'OneClickRun/master/res/appdata/pyload/pyload.conf'
  db = 'https://github.com/biplobsd/OneClickRun/raw/master/' \
        'res/appdata/pyload/files.db'
  urllib.request.urlretrieve(urlF, 'tools/pyload.zip')
  urllib.request.urlretrieve(conf, 'tools/pyload/pyload.conf')
  urllib.request.urlretrieve(db, 'tools/pyload/files.db')
  with zipfile.ZipFile('tools/pyload.zip', 'r') as zip_ref:
      zip_ref.extractall('tools/pyload')
  try:
    pathlib.Path('tools/pyload.zip').unlink()
  except FileNotFoundError:
    pass

  # runSh("apt install -y python-pycurl python-qt4 tesseract-ocr libtesseract-dev")
  # runSh("pip2 install pycrypto pyOpenSSL Jinja2 tesseract tesseract-ocr")

  # Using Threading to install
  threadRun = [
    threading.Thread(target=runSh, args=("apt install -y python-pycurl python-qt4 tesseract-ocr libtesseract-dev",)),
    threading.Thread(target=runSh, args=("pip2 install pycrypto pyOpenSSL Jinja2 tesseract tesseract-ocr",))
  ]
  for perTh in threadRun:perTh.start()
  for perTh in threadRun:perTh.join()

if not findProcess("python2.7", "pyLoadCore.py"):
  runCmd = "python2.7 /content/tools/pyload/pyload-stable/pyLoadCore.py" \
          " --configdir=/content/tools/pyload" \
          " --no-remote" \
          " --daemon"
  runSh(runCmd, shell=True)
# START_SERVER
# Ngrok region 'us','eu','ap','au','sa','jp','in'
clear_output()
Server = PortForward_wrapper(
    PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['rcloneConfig', 53682, 'http'], 
                            ['pyload', 8000, 'http']], REGION.lower(), 
    [f"{HOME}/.ngrok2/rclonePyload.yml", 4074]
).start('pyload')

## <img src='https://raw.githubusercontent.com/biplobsd/OneClickRun/master/img/spotify-downloader.png' height="25" alt="spotify-downloader" /> spotify-downloader

In [ ]:
#@markdown <br><center><img src='https://raw.githubusercontent.com/biplobsd/OneClickRun/master/img/spotify-downloader.png' height="100" alt="spotify-downloader"/></center>
#@markdown <center><h3>spotify-downloader<br />Download Spotify playlists from YouTube with albumart and meta-tags</h3></center><br>

import os, uuid, urllib.parse, re
import ipywidgets as widgets

from glob import glob
from urllib.parse import urlparse, parse_qs
from IPython.display import HTML, clear_output, YouTubeVideo
from IPython.utils.io import ask_yes_no
from google.colab import output, files


os.makedirs('tools/spotify-downloader/', exist_ok=True)
os.makedirs('downloads', exist_ok=True)
LATEST_VERSION = False #@param {type:"boolean"}

SET_VERSION = "2.0.8" #@param {type:"string"}
#@markdown  <font size='2px'>`https://pypi.org/project/spotdl/#history`</font>
# # Config files
# data = """spotify-downloader:
#   avconv: false
#   download-only-metadata: false
#   dry-run: false
#   file-format: '{artist} - {track_name}'
#   folder: /home/user/Music
#   input-ext: .m4a
#   log-level: INFO
#   manual: false
#   music-videos-only: false
#   no-fallback-metadata: false
#   no-metadata: false
#   no-spaces: false
#   output-ext: .mp3
#   overwrite: prompt
#   search-format: '{artist} - {track_name} lyrics'
#   skip: null
#   spotify_client_id: 4fe3fecfe5334023a1472516cc99d805
#   spotify_client_secret: 0f02b7c483c04257984695007a4a8d5c
#   trim-silence: false
#   write-successful: null
#   write-to: null
#   youtube-api-key: null
# """
# with open('tools/spotify-downloader/config.yml', 'w') as wnow:
#   wnow.write(data)

Links = widgets.Textarea(placeholder='''Link list
(one link per line)''')

fileFormat = widgets.Text(
    value='{artist} - {track_name}',
    placeholder='File name format',
    description="""File Name : file format to save the downloaded track with, each
                        tag is surrounded by curly braces. Possible formats:
                        ['track_name', 'artist', 'album', 'album_artist',
                        'genre', 'disc_number', 'duration', 'year',
                        'original_date', 'track_number', 'total_tracks',
                        'isrc']""",
    disabled=False
)

searchFormat = widgets.Text(
    value='{artist} - {track_name} lyrics',
    placeholder='Search format',
    description="""Search Format : search format to search for on YouTube, each tag is
                        surrounded by curly braces. Possible formats:
                        ['track_name', 'artist', 'album', 'album_artist',
                        'genre', 'disc_number', 'duration', 'year',
                        'original_date', 'track_number', 'total_tracks',
                        'isrc']""",
    disabled=False
)

tab = widgets.Tab()

LinksType = widgets.RadioButtons(
    options=['Songs', 'Playlist', 'Album', 'Username', 'Artist'],
    value='Songs',
    layout={'width': 'max-content'},
    description='Links type:',
    disabled=False,
)

SavePathYT = widgets.Dropdown(options=["/content/downloads", "/content"])

Extension = widgets.Select(options=["flac", "mp3", "m4a"], value="mp3")

TrimSilence = widgets.ToggleButton(
    value=False,
    description='Trim silence',
    disabled=False,
    button_style='',
    tooltip='remove silence from the start of the audio',
    icon='check'
)

writeM3u = widgets.ToggleButton(
    value=False,
    description='Write .m3u playlist',
    disabled=False,
    button_style='',
    tooltip='''generate an .m3u playlist file with youtube links
               given a text file containing tracks''',
    icon='check'
)

noMeta = widgets.ToggleButton(
    value=False,
    description='No metadata',
    disabled=False,
    button_style='',
    tooltip='do not embed metadata in tracks',
    icon='check'
)

noEnco = widgets.ToggleButton(
    value=False,
    description='No Encoding',
    disabled=False,
    button_style='',
    tooltip='do not encode media using FFmpeg',
    icon='check'
)

overwrite = widgets.Dropdown(
    disabled=False,
    options=['prompt', 'force', 'skip'],
    value='skip',
    description='Overwrite:',
)

quality = widgets.Dropdown(
    disabled=False,
    options=['worst', 'best'],
    value='best',
    description='Quality:',
)

inputExt = widgets.Dropdown(
    disabled=False,
    options=['automatic', 'm4a', 'opus'],
    value='automatic',
    description='Input format:',
)

logLevel = widgets.Dropdown(
    disabled=False,
    options=['INFO', 'WARNING', 'ERROR', 'DEBUG'],
    value='INFO',
    description='Log level:',
)

nf = widgets.ToggleButton(
    value=False,
    description='No fallback metadata',
    disabled=False,
    button_style='',
    tooltip='''do not use YouTube as fallback for metadata if track
                        not found on Spotify''',
    icon='check'
)

dryRun = widgets.ToggleButton(
    value=False,
    description='Dry run',
    disabled=False,
    button_style='',
    tooltip=''' show only track title and YouTube URL, and then skip
                        to the next track (if any)''',
    icon='check'
)

MusicVidOnly = widgets.ToggleButton(
    value=False,
    description='Music Videos Only',
    disabled=False,
    button_style='',
    tooltip='''search only for music videos on Youtube (works only
                        when YouTube API key is set''',
    icon='check'
)

NoSpaces = widgets.ToggleButton(
    value=False,
    description='No Spaces',
    disabled=False,
    button_style='',
    tooltip='''replace spaces with underscores in file names''',
    icon='check'
)

manual = widgets.ToggleButton(
    value=False,
    description='manually',
    disabled=False,
    button_style='',
    tooltip='''choose the track to download manually from a list of
                        matching tracks''',
    icon='check'
)

nr = widgets.ToggleButton(
    value=False,
    description='Keep original',
    disabled=False,
    button_style='',
    tooltip='''do not remove the original file after conversion''',
    icon='check'
)

ExtraArg = widgets.Text(placeholder="Extra Arguments")

class MakeButton(object):
  def __init__(self, title, callback, style):
    self._title = title
    self._callback = callback
    self._style = style
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    if self._style != "":
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button mod-" + self._style
    else:
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button"
    template = """<button class="{style_html}" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id, style_html=style_html)
    return html
  
def MakeLabel(description, button_style):
  return widgets.Button(description=description, disabled=True, button_style=button_style)

def RefreshPathYT():
  if os.path.exists("/content/drive/"):
    if os.path.exists("/content/drive/Shared drives/"):
      SavePathYT.options = ["/content", "/content/downloads", "/content/drive/My Drive"] + glob("/content/drive/My Drive/*/") + glob("/content/drive/Shared drives/*/")
    else:
      SavePathYT.options = ["/content", "/content/downloads", "/content/drive/My Drive"] + glob("/content/drive/My Drive/*/")
  else:
    SavePathYT.options = ["/content/downloads", "/content"]


def ShowYT():
  clear_output(wait=True)
  RefreshPathYT()
  mainTab = widgets.Box([
                        widgets.HBox([widgets.VBox([
                                      widgets.HTML("<b style=\"color:#888888;\">Link:</b>"), Links,
                                      LinksType, searchFormat, fileFormat, 
                                      widgets.HBox([TrimSilence, writeM3u, noMeta]),
                                      widgets.HBox([nf, dryRun, MusicVidOnly]),
                                      widgets.HBox([NoSpaces, manual, nr]),
                                      widgets.HBox([noEnco])
                                      ]),
                        widgets.VBox([
                                      widgets.HTML("<b style=\"color:#888888;\">Extension:</b>"), 
                                      Extension,
                                      widgets.HTML("<b style=\"color:#888888;\">Extra Arguments:</b>"), 
                                      ExtraArg, overwrite, quality, inputExt, 
                                      logLevel])
                        ])])
  
  tab.children = [mainTab]
  tab.set_title(0, 'spotify-downloader')
  display(HTML("<style>widget-tab-bar {display: none;}</style>"))
  display(tab)
  display(HTML("<h4 style=\"color:#888888;\">Save Location:</h4>"), SavePathYT, MakeButton("Refresh", RefreshPathYT, ""))
  if not os.path.exists("/content/drive/"):
    display(HTML("*If you want to save in Google Drive please run the cell below."))
  display(HTML("<br>"), MakeButton("Download", DownloadYT, "info"))

def DownloadYT():
  if Links.value.strip():
    Count = 0
    Total = str(len(Links.value.splitlines()))
    
    if writeM3u.value:
      M3u = '--write-m3u'
    else:
      M3u = ''
    
    if TrimSilence.value:
      trmS = '--trim-silence'
    else:
      trmS = ''
    
    if noMeta.value:
      noM = '--no-metadata'
    else:
      noM = ''
    
    if noEnco.value:
      noE = '--no-encode'
    else:
      noE = ''

    if not overwrite.disabled:
      oW = f'--overwrite {overwrite.value}'
    else:
      oW = ''

    if not quality.disabled:
      qL = f'--quality {quality.value}'
    else:
      qL = ''
    
    if not inputExt.disabled:
      iE = f'--input-ext {inputExt.value}'
    else:
      iE = ''

    if not logLevel.disabled:
      lL = f'--log-level {logLevel.value}'
    else:
      lL = ''

    if nf.value:
      nfv = '--no-fallback-metadata'
    else:
      nfv = ''
    
    if dryRun.value:
      drR = '--dry-run'
    else:
      drR = ''
    
    if MusicVidOnly.value:
      MsV = '--music-videos-only'
    else:
      MsV = ''
    
    if NoSpaces.value:
      NoS = '--no-spaces'
    else:
      NoS = ''
    
    if manual.value:
      mal = '--manual'
    else:
      mal = ''
    
    if nr.value:
      nro = '--no-remove-original' 
    else:
      nro = ''
    
    if not searchFormat.value == '{artist} - {track_name} lyrics':
      seFor = f'--search-format "{searchFormat.value}"'
    else:
      seFor = ''
    
    if not fileFormat.value == '{artist} - {track_name}':
      fiFor = f'--file-format "{fileFormat.value}"'
    else:
      fiFor = ''
    
    if not LinksType.value == 'Songs':
      with open('tools/spotify-downloader/finish.txt', 'a+') as master:
        for Link in Links.value.splitlines():
          if LinksType.value == 'Playlist':
            outFileName = !spotdl --playlist $Link
          elif LinksType.value == 'Album':
            outFileName = !spotdl --album $Link
          elif LinksType.value == 'Username':
            outFileName = !spotdl -u $Link
          elif LinksType.value == 'Artist':
            outFileName = !spotdl --all-albums $Link
          filename = re.search(r"to\s(.+\.txt)", outFileName[-1]).group(1)
          with open(filename, 'r') as r:
            master.write(r.read())
    else:
      for Link in Links.value.splitlines():
        with open('tools/spotify-downloader/finish.txt', 'w') as master:
          master.write(Link)
    # Extra Arguments
    
    extraargC = ExtraArg.value
    cmd = r"spotdl -l 'tools/spotify-downloader/finish.txt' " \
      fr"-f '{SavePathYT.value}' " \
      fr"-o {Extension.value} " \
      f"{seFor} {fiFor} " \
      f"{M3u} {trmS} {oW} {noM} {noE} {nfv} {drR} {MsV} {NoS} {mal} {nro}" \
      f"{qL} {iE} {lL}" 
    !$cmd
    ShowYT()

if not os.path.isfile("/usr/local/bin/spotdl"):
  get_ipython().system_raw(f"pip3 install spotdl{'=='+SET_VERSION if LATEST_VERSION else ''} && apt -y install ffmpeg")

ShowYT()

## <img src='http://pic2.orsoon.com/2017/0719/20170719091439708.png' height="60" alt="aria2" />___aria2___

In [ ]:
#@markdown <h3>⬅️ Click Here to START server</h3>
#@markdown <br><center><img src='http://pic2.orsoon.com/2017/0719/20170719091439708.png' height="200" alt="aria2"/></center>
#@markdown <center><h3>aria2<br />The next generation download utility.</h3></center><br>
import os, pathlib, zipfile, re
import urllib.request, requests
from IPython.display import HTML, clear_output
from urllib.parse import urlparse

#####################################
USE_FREE_TOKEN = True  # @param {type:"boolean"}
REGION = "JP" #@param ["US", "EU", "AP", "AU", "SA", "JP", "IN"]
# Aria2 Service
Aria2_rpc = True  # @param {type:"boolean"}
Ariang_WEBUI = True  # @param {type:"boolean"}
TOKEN = ""  # @param {type:"string"}
PORT_FORWARD = "argotunnel" #@param ["ngrok", "localhost", "argotunnel"]
# OUTPUT_DIR = ""  # @param {type:"string"}
PORT = 8221
HOME = os.path.expanduser("~")

if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")

from ocr import (
    runSh,
    loadingAn,
    PortForward_wrapper,
    CWD,
    displayUrl,
    findProcess,
    findPackageR
)

loadingAn()

# Ariang SETUP
runSh('apt install -y aria2')
pathlib.Path('ariang').mkdir(mode=0o777, exist_ok=True)
pathlib.Path('downloads').mkdir(mode=0o777, exist_ok=True)

# github latest releases tag define
def latestTag(link):
  import re
  from urllib.request import urlopen
  htmlF = urlopen(link+"/releases/latest").read().decode('UTF-8')
  return re.findall(r'.+\/tag\/([.0-9A-Za-z]+)".+/', htmlF)[0]

# Downloading latest version of ariang
if not os.path.exists("ariang/index.html"):
  # BASE_URL = r"https://github.com/mayswind/AriaNg"
  # LATEST_TAG = latestTag(BASE_URL)
  # urlF = f'{BASE_URL}/releases/download/{LATEST_TAG}/' \
  #             f'AriaNg-{LATEST_TAG}-AllInOne.zip'
  urllib.request.urlretrieve(
      findPackageR('mayswind/AriaNg', 'AllInOne.zip'),
      'ariang/new.zip')
  with zipfile.ZipFile('ariang/new.zip', 'r') as zip_ref:
      zip_ref.extractall('ariang')
  try:
    pathlib.Path('ariang/new.zip').unlink()
  except FileNotFoundError:
    pass

# START_ARIANG_WEBUI_AND_ARIA2_RPC
try:
  if not OUTPUT_DIR:
    OUTPUT_DIR = f"downloads/"
  elif not os.path.exists(OUTPUT_DIR):
    clear_output()
    print("Error: Your set path not found! Create path!")
    exx()
except:
  OUTPUT_DIR = f"{CWD}/downloads/"
if Aria2_rpc:
  if not findProcess("aria2c", "--enable-rpc"):
    try:
      trackers = requests.get(
          "https://trackerslist.com/best_aria2.txt"
      ).text
      cmdC = r"aria2c --enable-rpc --rpc-listen-port=6800 -D " \
            fr"-d {OUTPUT_DIR} " \
            r"-j 20 " \
            r"-c " \
            fr"--bt-tracker={trackers} " \
            r"--bt-request-peer-speed-limit=0 " \
            r"--bt-max-peers=0 " \
            r"--seed-ratio=0.0 " \
            r"--max-connection-per-server=10 " \
            r"--min-split-size=10M " \
            r"--follow-torrent=mem " \
            r"--disable-ipv6=true " \
            r" &"
      runSh(
          cmdC,
          shell=True
      )
    except:
      print("aria2 rpc not enabling. Try again later!")

# START_SERVER
# Ngrok region 'us','eu','ap','au','sa','jp','in'
clear_output()
if Aria2_rpc:
  Server = PortForward_wrapper(
    PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['Aria2_rpc', 6800, 'http']], REGION.lower(), 
    [f"{HOME}/.ngrok2/aria2.yml", 5042]
)
  data = Server.start('Aria2_rpc', displayB=False)
  Host = urlparse(data['url']).hostname
  port = "80"
  clear_output()
if Ariang_WEBUI:
  if Aria2_rpc:
    filePath = 'ariang/index.html'
    with open(filePath, 'r+') as f:
      read_data = f.read()
      f.seek(0)
      f.truncate(0)
      read_data = re.sub(r'(rpcHost:"\w+.")|rpcHost:""', 
                        f'rpcHost:"{Host}"', read_data)
      read_data = re.sub(r'protocol:"\w+."', r'protocol:"ws"', read_data)
      read_data = re.sub(r'rpcPort:"\d+."', f'rpcPort:"{port}"', read_data)
      f.write(read_data)
  try:
    urllib.request.urlopen(f"http://localhost:{PORT}")
  except:
    runSh(f"python3 -m http.server {PORT} &", shell=True, cd="ariang/")
  
  Server = PortForward_wrapper(
    PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['Ariang', PORT, 'http']], REGION.lower(), 
    [f"{HOME}/.ngrok2/ariang.yml", 5043]
)
  data2 = Server.start('Ariang', displayB=False)
  data2['url'] = urlparse(data2['url'])._replace(scheme='http').geturl()
  displayUrl(data2, pNamU='Ariang : ')

if Aria2_rpc:
  display(HTML("""<style>@import url('https://fonts.googleapis.com/css?family=Source+Code+Pro:200,900');  :root {   --text-color: hsla(210, 50%, 85%, 1);   --shadow-color: hsla(210, 40%, 52%, .4);   --btn-color: hsl(210, 80%, 42%);   --bg-color: #141218; }  * {   box-sizing: border-box; } button { position:relative; padding: 10px 20px;     border: none;   background: none;      font-family: "Source Code Pro";   font-weight: 900;font-size: 100%;     color: var(--text-color);      background-color: var(--btn-color);   box-shadow: var(--shadow-color) 2px 2px 22px;   border-radius: 4px;    z-index: 0;overflow: hidden; -webkit-user-select: text;-moz-user-select: text;-ms-user-select: text;user-select: text;}  button:focus {   outline-color: transparent;   box-shadow: var(--btn-color) 2px 2px 22px; }  .right::after, button::after {   content: var(--content);   display: block;   position: absolute;   white-space: nowrap;   padding: 40px 40px;   pointer-events:none; }  button::after{   font-weight: 200;   top: -30px;   left: -20px; }   .right, .left {   position: absolute;   width: 100%;   height: 100%;   top: 0; } .right {   left: 66%; } .left {   right: 66%; } .right::after {   top: -30px;   left: calc(-66% - 20px);      background-color: var(--bg-color);   color:transparent;   transition: transform .4s ease-out;   transform: translate(0, -90%) rotate(0deg) }  button:hover .right::after {   transform: translate(0, -47%) rotate(0deg) }  button .right:hover::after {   transform: translate(0, -50%) rotate(-7deg) }  button .left:hover ~ .right::after {   transform: translate(0, -50%) rotate(7deg) }  /* bubbles */ button::before {   content: '';   pointer-events: none;   opacity: .6;   background:     radial-gradient(circle at 20% 35%,  transparent 0,  transparent 2px, var(--text-color) 3px, var(--text-color) 4px, transparent 4px),     radial-gradient(circle at 75% 44%, transparent 0,  transparent 2px, var(--text-color) 3px, var(--text-color) 4px, transparent 4px),     radial-gradient(circle at 46% 52%, transparent 0, transparent 4px, var(--text-color) 5px, var(--text-color) 6px, transparent 6px);    width: 100%;   height: 300%;   top: 0;   left: 0;   position: absolute;   animation: bubbles 5s linear infinite both; }  @keyframes bubbles {   from {     transform: translate();   }   to {     transform: translate(0, -66.666%);   } }.zui-table {    border: solid 1px #DDEEEE;    border-collapse: collapse;    border-spacing: 0;    font: normal 13px;}.zui-table thead th {    background-color: #DDEFEF;    border: solid 1px #DDEEEE;    color: #0000009e;    padding: 10px;    text-align: left;}.zui-table tbody td {border: solid 1px #effff97a;color: #ffffffd1;    padding: 10px;}</style><center><button><table class="zui-table blueBG"><p>Aria2 rpc config<p><thead>        <tr>            <th>Protocol</th>            <th>Host</th>            <th>Port</th>        </tr>    </thead>    <tbody>        <tr>            <td>WebSocket</td><td>"""+Host+"""</td><td>"""+port+"""</td></tr></tbody></table><a target="_blank" style="text-decoration: none;color: hsla(210, 50%, 85%, 1);font-size: 10px;" href="https://raw.githubusercontent.com/biplobsd/OneClickRun/master/img/aria2RpcConfigSetup.gif">NB. How to setup this's config. [Click ME]</a></button><center>"""))


___Auto share link supported : Disk.yandex.com, Gdrive and youtube-dl all extractor...___

In [ ]:
URL = "" #@param {type:"string"}
#@markdown <center><h5>Default output path /content/downloads/</h5></center>
OUTPUT_PATH = "" #@param {type:"string"}

import pathlib
import shutil
import hashlib
import requests
from urllib.parse import urlparse
from os import path, mkdir
if not path.exists("/root/.ipython/ocr.py"): 
    from subprocess import run
    from shlex import split

    shellCmd = "wget -qq https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py \
                    -O /root/.ipython/ocr.py"
    run(split(shellCmd))
from ocr import runSh

def youtubedlInstall():
  if not path.isfile("/usr/local/bin/youtube-dl"):
    cmdC = "rm -rf /content/sample_data/ && " \
            " mkdir -p -m 666 /root/.YouTube-DL/ &&" \
            " apt-get install atomicparsley &&" \
            " curl -L https://yt-dl.org/downloads/latest/youtube-dl " \
            "-o /usr/local/bin/youtube-dl &&" \
            " chmod a+rx /usr/local/bin/youtube-dl"
    get_ipython().system_raw(cmdC)

def aria2Install():
  runSh('apt install -y aria2')

def istmd(URL): 
  link = urlparse(URL)
    
  #YandexDisk
  if link.netloc == "yadi.sk":
    API_ENDPOINT = 'https://cloud-api.yandex.net/v1/disk/public/resources/' \
                    '?public_key={}&path=/{}&offset={}'
    dry = False
    def md5sum(file_path):
        md5 = hashlib.md5()
        with open(file_path, 'rb') as f:
            for chunk in iter(lambda: f.read(128 * md5.block_size), b''):
                md5.update(chunk)
        return md5.hexdigest()


    def check_and_download_file(target_path, url, size, checksum):
        if path.isfile(target_path):
            if size == path.getsize(target_path):
                if checksum == md5sum(target_path):
                    print('URL {}'.format(url))
                    print('skipping correct {}'.format(target_path))
                    return
        if not dry:
            print('URL {}'.format(url))
            print('downloading {}'.format(target_path))
            runSh(f'aria2c -x 16 -s 16 -k 1M -d {OUTPUT_PATH} {url}', output=True)
            # r = requests.get(url, stream=True)
            # with open(target_path, 'wb') as f:
            #     shutil.copyfileobj(r.raw, f)


    def download_path(target_path, public_key, source_path, offset=0):
        print('getting "{}" at offset {}'.format(source_path, offset))
        current_path = path.join(target_path, source_path)
        pathlib.Path(current_path).mkdir(parents=True, exist_ok=True)
        jsn = requests.get(API_ENDPOINT.format(public_key, source_path, offset)).json()
        def try_as_file(j):
            if 'file' in j:
                file_save_path = path.join(current_path, j['name'])
                check_and_download_file(file_save_path, j['file'], j['size'], j['md5'])
                return True
            return False

        # first try to treat the actual json as a single file description
        if try_as_file(jsn):
            return

        # otherwise treat it as a directory
        emb = jsn['_embedded']
        items = emb['items']
        for i in items:
            # each item can be a file...
            if try_as_file(i):
                continue
            # ... or a directory
            else:
                subdir_path = path.join(source_path, i['name'])
                download_path(target_path, public_key, subdir_path)

        # check if current directory has more items
        last = offset + emb['limit']
        if last < emb['total']:
            download_path(target_path, public_key, source_path, last)
    download_path(OUTPUT_PATH, URL, '')
    return False  
  return URL

if not OUTPUT_PATH:
  OUTPUT_PATH = "/content/downloads/"
  
if not URL == "":
  aria2Install()
  youtubedlInstall()
  try:
    mkdir("downloads")
  except FileExistsError:
    pass
  url = istmd(URL)
  if url != False:
    print('URL {}'.format(URL))
    cmdC = f'youtube-dl -o "{OUTPUT_PATH}/%(title)s" {URL} ' \
            '--external-downloader aria2c ' \
            '--external-downloader-args "-x 16 -s 16 -k 1M"'
    runSh(cmdC, output=True)
else:
  print("Please input url")

##<img src='https://raw.githubusercontent.com/biplobsd/OneClickRun/master/img/zip.png' height="30" alt="netdata"> __Extracte/Archive Zip/Tar/Rar__

In [ ]:
# ============================= FORM ============================= #
# @markdown #### ⬅️ Extract Files
MODE = "7Z"  # @param ["UNZIP", "UNTAR", "UNRAR", "7Z"]
PATH_TO_FILE = ""  # @param {type:"string"}
extractPath = ""  # @param {type:"string"}
ARCHIVE_PASSWORD = "" #@param {type:"string"}

# ================================================================ #
import os, urllib.request
HOME = os.path.expanduser("~")

if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")

from ocr import (
    runSh,
    checkAvailable,
)

def extractFiles():
    global extractPath
    if ARCHIVE_PASSWORD:
      passADD = f'-P {ARCHIVE_PASSWORD}'
    else:
      passADD = ''
    if not extractPath:
      extractPath = "/content/extract"
    os.makedirs(extractPath, exist_ok=True)
    if MODE == "UNZIP":
        runSh('unzip '+passADD+f' "{PATH_TO_FILE}" -d "{extractPath}"', output=True)
    elif MODE == "UNRAR":
        runSh(f'unrar x "{PATH_TO_FILE}" "{extractPath}" '+passADD+' -o+', output=True)
    elif MODE == "UNTAR":
        runSh(f'tar -C "{extractPath}" -xvf "{PATH_TO_FILE}"', output=True)
    else:
        runSh(f'7z x "{PATH_TO_FILE}" -o{extractPath} '+passADD, output=True)


extractFiles()

In [ ]:
# ============================= FORM ============================= #
# @markdown #### ⬅️ Archive Files
MODE = "ZIP" #@param ["ZIP", "TAR", "7Z"]
FILENAME = ""  # @param {type:"string"}
PATH_TO_FILE = ""  # @param {type:"string"}
ARCHIVE_PASSWORD = "" #@param {type:"string"}

# option supports b k m g (bytes, kilobytes, megabytes, gigabytes)
SPLIT = "no" #@param ["1g", "2g", "3g", "4g", "5g", "no"]


compress = 4#@param  {type:"slider", min:0, max:9, step:0}
#@markdown <font size=2%>Use this to path separate between to this character `|`. Ex `path/to /1 | path/to/2` </br>#Split only in zip mode.(Only for noobs)
# ================================================================ #
from pathlib import PurePosixPath

pathList = PATH_TO_FILE.split('|')
if MODE == "ZIP":
    if not FILENAME:
      FILENAME = "/content/NEW_FILE.ZIP"
    if ARCHIVE_PASSWORD:
      passADD = f'--password "{ARCHIVE_PASSWORD}"'
    else:
      passADD = ''
    splitC = f"-s {SPLIT}" if not 'no' in SPLIT else "" 
    for part in pathList:
      pathdic = PurePosixPath(part.strip())
      parent = pathdic.parent
      partName = pathdic.parts[-1]
      cmd = f'cd "{parent}" && zip {passADD} -{compress} {splitC} -v -r -u "{FILENAME}" "{partName}"'
      !$cmd
elif MODE == "TAR":
    if not FILENAME:
      FILENAME = "/content/NEW_FILE"
    cmd = f'GZIP=-{compress} tar -zcvf "{FILENAME}.tar.gz" {PATH_TO_FILE}'
    !$cmd
else:
    if not FILENAME:
        FILENAME = "/content/NEW_FILE"
    for part in pathList:
      pathdic = PurePosixPath(part.strip())
      parent = pathdic.parent
      partName = pathdic.parts[-1]
      cmd = f'cd "{parent}" && 7z a -mx={compress} "{FILENAME}.7z" "{partName}"'
      !$cmd


#`transferTools(`<font size="3x" color="#ee0e15">`Mega` </font>,<font size="3px" color="#3f79ad"> `Rclone`</font>`)`

##![alt text](https://mega.nz/favicon.ico?v=3) __TRANSFERE to MEGA__





In [ ]:
# @markdown <center><h2>INPUT YOUR MEGA ID</h2></center><br>
from functools import wraps
import errno
import os
import signal
import subprocess
import shlex


class TimeoutError(Exception):
    pass


def timeout(seconds=10, error_message=os.strerror(errno.ETIME)):
    def decorator(func):
        def _handle_timeout(signum, frame):
            raise TimeoutError(error_message)

        def wrapper(*args, **kwargs):
            signal.signal(signal.SIGALRM, _handle_timeout)
            signal.alarm(seconds)
            try:
                result = func(*args, **kwargs)
            finally:
                signal.alarm(0)
            return result

        return wraps(func)(wrapper)

    return decorator


if not os.path.exists("/root/.ipython/ocr.py"):
    from subprocess import run
    from shlex import split

    shellCmd = "wget -qq https://raw.githubusercontent.com/biplobsd/OneClickRun/master/res/ocr.py \
                    -O /root/.ipython/ocr.py"
    run(split(shellCmd))
from ocr import runSh

@timeout(10)
def runShT(args):
    return runSh(args, output=True)


# MEGAcmd installing
if not os.path.exists("/usr/bin/mega-cmd"):
    print("Installing MEGA ...")
    runSh('sudo apt-get -y update')
    runSh('sudo apt-get -y install libmms0 libc-ares2 libc6 libcrypto++6 libgcc1 libmediainfo0v5 libpcre3 libpcrecpp0v5 libssl1.1 libstdc++6 libzen0v5 zlib1g apt-transport-https')
    runSh('sudo curl -sL -o /var/cache/apt/archives/MEGAcmd.deb https://mega.nz/linux/MEGAsync/Debian_9.0/amd64/megacmd-Debian_9.0_amd64.deb', output=True)
    runSh('sudo dpkg -i /var/cache/apt/archives/MEGAcmd.deb', output=True)
    print("MEGA is installed.")
else:
    !pkill mega-cmd

# INPUT YOUR MEGA ID

USERNAME = ""  # @param {type:"string"}
PASSWORD = ""  # @param {type:"string"}
if not (USERNAME == "" or PASSWORD == ""):
    try:
        runShT(f"mega-login {USERNAME} {PASSWORD}")
    except TimeoutError:
        runSh('mega-whoami', output=True)
else:
    print("Please Input your Mega IDs.")


In [ ]:
import time
import subprocess
import contextlib
from IPython.display import clear_output
# @markdown <br><center><img src='https://www.impactcomputing.co.uk/uploads/files/services/managed-office-365.svg' height="100" alt="netdata"/></center>
# @markdown <center><h2>TRANSFERE NOW</h2></center><br>
# TRANSFERE NOW

PATH_TO_FILE = ""  # @param {type:"string"}
# Unix, Windows and old Macintosh end-of-line
newlines = ['\n', '\r\n', '\r']


def unbuffered(proc, stream='stdout'):
    stream = getattr(proc, stream)
    with contextlib.closing(stream):
        while True:
            out = []
            last = stream.read(1)
            # Don't loop forever
            if last == '' and proc.poll() is not None:
                break
            while last not in newlines:
                # Don't loop forever
                if last == '' and proc.poll() is not None:
                    break
                out.append(last)
                last = stream.read(1)
            out = ''.join(out)
            yield out


def transfare():
    cmd = ['mega-put', PATH_TO_FILE, '/colab']
    proc = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        # Make all end-of-lines '\n'
        universal_newlines=True,
    )
    for line in unbuffered(proc):
        clear_output(wait=True)
        print(line)

try:
    transfare()
except FileNotFoundError:
    print("Login your account!")


In [ ]:
import sys, os, urllib.request
import time
import subprocess
import contextlib
from IPython.display import clear_output

HOME = os.path.expanduser("~")
if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")

from ocr import (
    runSh,
    loadingAn,
)
#@title MEGA public link download
URL = "" #@param {type:"string"}
OUTPUT_PATH = "" #@param {type:"string"}
if not OUTPUT_PATH:
  os.makedirs("downloads", exist_ok=True)
  OUTPUT_PATH = "downloads"
# MEGAcmd installing
if not os.path.exists("/usr/bin/mega-cmd"):
    loadingAn()
    print("Installing MEGA ...")
    runSh('sudo apt-get -y update')
    runSh('sudo apt-get -y install libmms0 libc-ares2 libc6 libcrypto++6 libgcc1 libmediainfo0v5 libpcre3 libpcrecpp0v5 libssl1.1 libstdc++6 libzen0v5 zlib1g apt-transport-https')
    runSh('sudo curl -sL -o /var/cache/apt/archives/MEGAcmd.deb https://mega.nz/linux/MEGAsync/Debian_9.0/amd64/megacmd-Debian_9.0_amd64.deb', output=True)
    runSh('sudo dpkg -i /var/cache/apt/archives/MEGAcmd.deb', output=True)
    print("MEGA is installed.")
    clear_output()

# Unix, Windows and old Macintosh end-of-line
newlines = ['\n', '\r\n', '\r']

def unbuffered(proc, stream='stdout'):
    stream = getattr(proc, stream)
    with contextlib.closing(stream):
        while True:
            out = []
            last = stream.read(1)
            # Don't loop forever
            if last == '' and proc.poll() is not None:
                break
            while last not in newlines:
                # Don't loop forever
                if last == '' and proc.poll() is not None:
                    break
                out.append(last)
                last = stream.read(1)
            out = ''.join(out)
            yield out


def transfare():
    import codecs
    decoder = codecs.getincrementaldecoder("UTF-8")()
    cmd = ["mega-get", URL, OUTPUT_PATH]
    proc = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        # Make all end-of-lines '\n'
        universal_newlines=True,
    )
    for line in unbuffered(proc):
        print(line)
        


transfare()



## <img src='https://biplobsd.github.io/RLabClone/img/title_rclonelab.svg' height="45" alt="RcloneLab"/>



In [ ]:
# ============================= FORM ============================= #
# @markdown #### ⬅️ Execute rClone

Mode = "Copy"  # @param ["Move", "Copy", "Sync", "Verify", "Dedupe", "Clean Empty Dirs", "Empty Trash"]
Source = ""  # @param {type:"string"}
Destination = ""  # @param {type:"string"}
Extra_Arguments = ""  # @param {type:"string"}
COPY_SHARED_FILES = False  # @param{type: "boolean"}
Compare = "Size & Checksum"
TRANSFERS, CHECKERS = 20, 20
THROTTLE_TPS = True
BRIDGE_TRANSFER = False  # @param{type: "boolean"}
FAST_LIST = False  # @param{type: "boolean"}
OPTIMIZE_GDRIVE = True
SIMPLE_LOG = True
RECORD_LOGFILE = False  # @param{type: "boolean"}
SKIP_NEWER_FILE = False
SKIP_EXISTED = False
SKIP_UPDATE_MODTIME = False
ONE_FILE_SYSTEM = False
LOG_LEVEL = "DEBUG"
SYNC_MODE = "Delete after transfering"
SYNC_TRACK_RENAME = True
DEDUPE_MODE = "Largest"
USE_TRASH = True
DRY_RUN = False  # @param{type: "boolean"}
# ================================================================ #
from os import path as _p

if not _p.exists("/root/.ipython/rlab_utils.py"):
    from shlex import split as _spl
    from subprocess import run

    shellCmd = "wget -qq https://biplobsd.github.io/RLabClone/res/rlab_utils.py \
                    -O /root/.ipython/rlab_utils.py"
    run(_spl(shellCmd))

from datetime import datetime as _dt
from rlab_utils import (
    displayOutput,
    checkAvailable,
    runSh,
    prepareSession,
    PATH_RClone_Config,
    accessSettingFile,
    memGiB,
)


def populateActionArg():
    if Mode == "Copy":
        actionArg = "copy"
    elif Mode == "Sync":
        actionArg = "sync"
    elif Mode == "Verify":
        actionArg = "check"
    elif Mode == "Dedupe":
        actionArg = "dedupe largest"
    elif Mode == "Clean Empty Dirs":
        actionArg = "rmdirs"
    elif Mode == "Empty Trash":
        actionArg = "delete"
    else:
        actionArg = "move"

    return actionArg


def populateCompareArg():
    if Compare == "Mod-Time":
        compareArg = "--ignore-size"
    elif Compare == "Size":
        compareArg = "--size-only"
    elif Compare == "Checksum":
        compareArg = "-c --ignore-size"
    else:
        compareArg = "-c"

    return compareArg


def populateOptimizeGDriveArg():
    return (
        "--buffer-size 256M \
            --drive-chunk-size 256M \
                --drive-upload-cutoff 256M \
                    --drive-acknowledge-abuse \
                        --drive-keep-revision-forever"
        if OPTIMIZE_GDRIVE
        else "--buffer-size 128M"
    )


def populateGDriveCopyArg():
    if BRIDGE_TRANSFER and memGiB() < 13:
        global TRANSFERS, CHECKERS
        TRANSFERS, CHECKERS = 10, 80
    else:
        pass
    return "--disable copy" if BRIDGE_TRANSFER else "--drive-server-side-across-configs"


def populateStatsArg():
    statsArg = "--stats-one-line --stats=5s" if SIMPLE_LOG else "--stats=5s -P"
    if LOG_LEVEL != "OFF":
        statsArg += " -v" if SIMPLE_LOG else "-vv"
    elif LOG_LEVEL == "INFO":
        statsArg += " --log-level INFO"
    elif LOG_LEVEL == "ERROR":
        statsArg += " --log-level ERROR"
    else:
        statsArg += " --log-level DEBUG"
    return statsArg


def populateSyncModeArg():
    if Mode != "Sync":
        return ""
    elif SYNC_MODE == "Delete before transfering":
        syncModeArg = "--delete-before"
    elif SYNC_MODE == "Delete after transfering":
        syncModeArg = "--delete-after"
    else:
        syncModeArg = "--delete-during"
    if SYNC_TRACK_RENAME:
        syncModeArg += " --track-renames"
    return syncModeArg


def populateDedupeModeArg():
    if DEDUPE_MODE == "Interactive":
        dedupeModeArg = "--dedupe-mode interactive"
    elif DEDUPE_MODE == "Skip":
        dedupeModeArg = "--dedupe-mode skip"
    elif DEDUPE_MODE == "First":
        dedupeModeArg = "--dedupe-mode first"
    elif DEDUPE_MODE == "Newest":
        dedupeModeArg = "--dedupe-mode newest"
    elif DEDUPE_MODE == "Oldest":
        dedupeModeArg = "--dedupe-mode oldest"
    elif DEDUPE_MODE == "Rename":
        dedupeModeArg = "--dedupe-mode rename"
    else:
        dedupeModeArg = "--dedupe-mode largest"

    return dedupeModeArg


def generateCmd():
    sharedFilesArgs = (
        "--drive-shared-with-me --files-from /content/upload.txt --no-traverse"
        if COPY_SHARED_FILES
        else ""
    )

    logFileArg = f"--log-file /content/rclone_log.txt -vv -P"

    args = [
        "rclone",
        f"--config {PATH_RClone_Config}/rclone.conf",
        '--user-agent "Mozilla"',
        populateActionArg(),
        f'"{Source}"',
        f'"{Destination}"' if Mode in ("Move", "Copy", "Sync") else "",
        f"--transfers {str(TRANSFERS)}",
        f"--checkers {str(CHECKERS)}",
    ]

    if Mode == "Verify":
        args.append("--one-way")
    elif Mode == "Empty Trash":
        args.append("--drive-trashed-only --drive-use-trash=false")
    else:
        args.extend(
            [
                populateGDriveCopyArg(),
                populateSyncModeArg(),
                populateCompareArg(),
                populateOptimizeGDriveArg(),
                "-u" if SKIP_NEWER_FILE else "",
                "--ignore-existing" if SKIP_EXISTED else "",
                "--no-update-modtime" if SKIP_UPDATE_MODTIME else "",
                "--one-file-system" if ONE_FILE_SYSTEM else "",
                "--tpslimit 95 --tpslimit-burst 40" if THROTTLE_TPS else "",
                "--fast-list" if FAST_LIST else "",
                "--delete-empty-src-dirs" if Mode == "Move" else "",
            ]
        )
    args.extend(
        [
            "-n" if DRY_RUN else "",
            populateStatsArg() if not RECORD_LOGFILE else logFileArg,
            sharedFilesArgs,
            Extra_Arguments,
        ]
    )

    return args


def executeRclone():
    prepareSession()
    if Source.strip() == "":
        displayOutput("❌ The Source field is empty.")
        return
    if checkAvailable("/content/rclone_log.txt"):
        if not checkAvailable("/content/logfiles"):
            runSh("mkdir -p -m 666 /content/logfiles")
        job = accessSettingFile("job.txt")
        runSh(
            f'mv /content/rclone_log.txt /content/logfiles/{job["title"]}_{job["status"]}_logfile.txt'
        )

    onGoingJob = {
        "title": f'{Mode}_{Source}_{Destination}_{_dt.now().strftime("%a-%H-%M-%S")}',
        "status": "ongoing",
    }
    accessSettingFile("job.txt", onGoingJob)

    cmd = " ".join(generateCmd())
    runSh(cmd, output=True)
    displayOutput(Mode, "success")

    onGoingJob["status"] = "finished"
    accessSettingFile("job.txt", onGoingJob)

executeRclone()


In [ ]:
#@markdown <center><h3>Rclone MOUNT / UNMOUNT</h3>Mount the remote as file system on a mountpoint.</center>
Cache_Directory = "DISK" #@param ["RAM", "DISK"]
import os
from IPython.display import HTML, clear_output
import uuid
import ipywidgets as widgets
from google.colab import output
import re
#####################################
if not os.path.exists("/root/.ipython/rlab_utils.py"):
  from shlex import split as _spl
  from subprocess import run

  shellCmd = "wget -qq https://biplobsd.github.io/RLabClone/res/rlab_utils.py \
                  -O /root/.ipython/rlab_utils.py"
  run(_spl(shellCmd))

from rlab_utils import (
    runSh,
    prepareSession,
    PATH_RClone_Config,
)

class MakeButton(object):
  def __init__(self, title, callback, style):
    self._title = title
    self._callback = callback
    self._style = style
  def _repr_html_(self):
    callback_id = 'button-' + str(uuid.uuid4())
    output.register_callback(callback_id, self._callback)
    if self._style != "":
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button mod-" + self._style
    else:
      style_html = "p-Widget jupyter-widgets jupyter-button widget-button"
    template = """<button class="{style_html}" id="{callback_id}">{title}</button>
        <script>
          document.querySelector("#{callback_id}").onclick = (e) => {{
            google.colab.kernel.invokeFunction('{callback_id}', [], {{}})
            e.preventDefault();
          }};
        </script>"""
    html = template.format(title=self._title, callback_id=callback_id, style_html=style_html)
    return html
  
def ShowAC():
  clear_output(wait=True)
  display(
      widgets.HBox(
          [widgets.VBox(
              [widgets.HTML(
                  '''<h3 style="font-family:Trebuchet MS;color:#4f8bd6;margin-top:0px;">
                  Rclone available config...</h3>
                  '''
                  ),
               mountNam]
               )
          ]
          )
      )
  
  display(HTML("<br>"), MakeButton("Mount", MountCMD, "primary"),
          MakeButton("Unmount", unmountCMD, "danger"))

prepareSession()
content = open(f"{PATH_RClone_Config}/rclone.conf").read()
avCon = re.findall(r"^\[(.+)\]$", content, re.M)
mountNam = widgets.Dropdown(options=avCon)

if Cache_Directory == 'RAM':
  cache_path = '/dev/shm'
elif Cache_Directory == 'DISK':
  os.makedirs('/tmp', exist_ok=True)
  cache_path = '/tmp'

def MountCMD():
    mPoint = f"/content/drives/{mountNam.value}"
    os.makedirs(mPoint, exist_ok=True)
    cmd = rf"rclone mount {mountNam.value}: {mPoint}" \
      rf" --config {PATH_RClone_Config}/rclone.conf" \
      ' --user-agent "Mozilla"' \
      ' --buffer-size 256M' \
      ' --transfers 10' \
      ' --vfs-cache-mode full' \
      ' --vfs-cache-max-age 0h0m1s' \
      ' --vfs-cache-poll-interval 0m1s' \
      f' --cache-dir {cache_path}' \
      ' --allow-other' \
      ' --daemon'

    if runSh(cmd, shell=True) == 0:
      print(f"Mount success! - \t{mPoint}")
    else:
      print(f"Mount failed! - \t{mPoint}")

def unmountCMD():
  mPoint = f"/content/drives/{mountNam.value}"
  if os.system(f"fusermount -uz {mPoint}") == 0:
    runSh(f"rm -r {mPoint}")
    print(f"Unmounted success! - \t{mPoint}")
  else:
    runSh(f"fusermount -uz {mPoint}", output=True)

ShowAC()

In [ ]:
#@markdown <center><h3>Rclone webui</h3></center><br>
import os, signal
import random
import string
import urllib.request
from IPython.display import HTML, clear_output
from urllib.parse import urlparse
import time

#####################################
USE_FREE_TOKEN = True  # @param {type:"boolean"}
TOKEN = ""  # @param {type:"string"}
REGION = "AP" #@param ["US", "EU", "AP", "AU", "SA", "JP", "IN"]
PORT_FORWARD = "argotunnel" #@param ["ngrok", "localhost", "argotunnel"]
#@markdown Default <br>&emsp;username : user<br>&emsp;password : pass
HOME = os.path.expanduser("~")
runW = get_ipython()

if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")

if not os.path.exists("/root/.ipython/rlab_utils.py"):
  from shlex import split as _spl
  from subprocess import run

  shellCmd = "wget -qq https://biplobsd.github.io/RLabClone/res/rlab_utils.py \
                  -O /root/.ipython/rlab_utils.py"
  run(_spl(shellCmd))

from ocr import (
    runSh,
    loadingAn,
    PortForward_wrapper,
    displayUrl,
    findProcess,
    CWD,
    textAn,
    checkAvailable
)
from rlab_utils import (
    displayOutput,
    checkAvailable,
    prepareSession,
    PATH_RClone_Config,
    accessSettingFile,
    memGiB,
)

loadingAn()
prepareSession()
pid = findProcess("rclone", "rcd", isPid=True)
try:
  os.kill(int(pid), signal.SIGTERM)
except TypeError:
  pass
cmd = "rclone rcd --rc-web-gui --rc-addr :5572" \
      " --rc-serve" \
      " --rc-user=user --rc-pass=pass" \
      " --rc-no-auth" \
      rf" --config {PATH_RClone_Config}/rclone.conf" \
      ' --user-agent "Mozilla"' \
      ' --transfers 16' \
      " &"
runSh(cmd, shell=True)
# START_SERVER
# Ngrok region 'us','eu','ap','au','sa','jp','in'
clear_output()
Server = PortForward_wrapper(
    PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['rclone', 5572, 'http'],
                            ['filebrowser', 4000, 'http']], REGION.lower(), 
    [f"{HOME}/.ngrok2/filebrowserRclone.yml", 4099]
).start('rclone', displayB=False)
# output
clear_output()
displayUrl(Server, pNamU='Rclone webUI : ', 
           ExUrl=fr"https://user:pass@{urlparse(Server['url']).hostname}")

In [ ]:
#@markdown <center><h3>Create/Edit Rclone config</h3>Create a new remote with name, type and options.<br><font size=1px>After created your config file download that. Next time just upload and you are done!</font></center>
import os, urllib.request
from IPython.display import HTML
USE_FREE_TOKEN = True  # @param {type:"boolean"}
TOKEN = ""  # @param {type:"string"}
REGION = "SA" #@param ["US", "EU", "AP", "AU", "SA", "JP", "IN"]
HOME = os.path.expanduser("~")
runW = get_ipython()

if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")

#####################################

if not os.path.exists("/root/.ipython/rlab_utils.py"):
  from shlex import split as _spl
  from subprocess import run

  shellCmd = "wget -qq https://biplobsd.github.io/RLabClone/res/rlab_utils.py \
                  -O /root/.ipython/rlab_utils.py"
  run(_spl(shellCmd))

from rlab_utils import (
    prepareSession,
    PATH_RClone_Config,
    runSh
)
from ocr import (
    PortForward_wrapper
)

###################################
import codecs
import contextlib
import locale
import os
import pty
import select
import signal
import subprocess
import sys
import termios
import time


from IPython.utils import text
import six
from google.colab import _ipython
from google.colab import _message
from google.colab.output import _tags

# Linux read(2) limits to 0x7ffff000 so stay under that for clarity.
_PTY_READ_MAX_BYTES_FOR_TEST = 2**20  # 1MB

_ENCODING = 'UTF-8'

class ShellResult(object):
  """Result of an invocation of the shell magic.

  Note: This is intended to mimic subprocess.CompletedProcess, but has slightly
  different characteristics, including:
    * CompletedProcess has separate stdout/stderr properties. A ShellResult
      has a single property containing the merged stdout/stderr stream,
      providing compatibility with the existing "!" shell magic (which this is
      intended to provide an alternative to).
    * A custom __repr__ method that returns output. When the magic is invoked as
      the only statement in the cell, Python prints the string representation by
      default. The existing "!" shell magic also returns output.
  """

  def __init__(self, args, returncode, command_output):
    self.args = args
    self.returncode = returncode
    self.output = command_output

  def check_returncode(self):
    if self.returncode:
      raise subprocess.CalledProcessError(
          returncode=self.returncode, cmd=self.args, output=self.output)

  def _repr_pretty_(self, p, cycle):  # pylint:disable=unused-argument
    # Note: When invoking the magic and not assigning the result
    # (e.g. %shell echo "foo"), Python's default semantics will be used and
    # print the string representation of the object. By default, this will
    # display the __repr__ of ShellResult. Suppress this representation since
    # the output of the command has already been displayed to the output window.
    if cycle:
      raise NotImplementedError


def _configure_term_settings(pty_fd):
  term_settings = termios.tcgetattr(pty_fd)
  # ONLCR transforms NL to CR-NL, which is undesirable. Ensure this is disabled.
  # http://man7.org/linux/man-pages/man3/termios.3.html
  term_settings[1] &= ~termios.ONLCR

  # ECHOCTL echoes control characters, which is undesirable.
  term_settings[3] &= ~termios.ECHOCTL

  termios.tcsetattr(pty_fd, termios.TCSANOW, term_settings)


def _run_command(cmd, clear_streamed_output):
  """Calls the shell command, forwarding input received on the stdin_socket."""
  locale_encoding = locale.getpreferredencoding()
  if locale_encoding != _ENCODING:
    raise NotImplementedError(
        'A UTF-8 locale is required. Got {}'.format(locale_encoding))

  parent_pty, child_pty = pty.openpty()
  _configure_term_settings(child_pty)

  epoll = select.epoll()
  epoll.register(
      parent_pty,
      (select.EPOLLIN | select.EPOLLOUT | select.EPOLLHUP | select.EPOLLERR))

  try:
    temporary_clearer = _tags.temporary if clear_streamed_output else _no_op

    with temporary_clearer(), _display_stdin_widget(
        delay_millis=500) as update_stdin_widget:
      # TODO(b/115531839): Ensure that subprocesses are terminated upon
      # interrupt.
      p = subprocess.Popen(
          cmd,
          shell=True,
          executable='/bin/bash',
          stdout=child_pty,
          stdin=child_pty,
          stderr=child_pty,
          close_fds=True)
      # The child PTY is only needed by the spawned process.
      os.close(child_pty)

      return _monitor_process(parent_pty, epoll, p, cmd, update_stdin_widget)
  finally:
    epoll.close()
    os.close(parent_pty)


class _MonitorProcessState(object):

  def __init__(self):
    self.process_output = six.StringIO()
    self.is_pty_still_connected = True


def _monitor_process(parent_pty, epoll, p, cmd, update_stdin_widget):
  """Monitors the given subprocess until it terminates."""
  state = _MonitorProcessState()

  # A single UTF-8 character can span multiple bytes. os.read returns bytes and
  # could return a partial byte sequence for a UTF-8 character. Using an
  # incremental decoder is incrementally fed input bytes and emits UTF-8
  # characters.
  decoder = codecs.getincrementaldecoder(_ENCODING)()

  num_interrupts = 0
  echo_status = None
  while True:
    try:
      result = _poll_process(parent_pty, epoll, p, cmd, decoder, state)
      if result is not None:
        return result
      term_settings = termios.tcgetattr(parent_pty)
      new_echo_status = bool(term_settings[3] & termios.ECHO)
      if echo_status != new_echo_status:
        update_stdin_widget(new_echo_status)
        echo_status = new_echo_status
    except KeyboardInterrupt:
      try:
        num_interrupts += 1
        if num_interrupts == 1:
          p.send_signal(signal.SIGINT)
        elif num_interrupts == 2:
          # Process isn't responding to SIGINT and user requested another
          # interrupt. Attempt to send SIGTERM followed by a SIGKILL if the
          # process doesn't respond.
          p.send_signal(signal.SIGTERM)
          time.sleep(0.5)
          if p.poll() is None:
            p.send_signal(signal.SIGKILL)
      except KeyboardInterrupt:
        # Any interrupts that occur during shutdown should not propagate.
        pass

      if num_interrupts > 2:
        # In practice, this shouldn't be possible since
        # SIGKILL is quite effective.
        raise


def _poll_process(parent_pty, epoll, p, cmd, decoder, state):
  """Polls the process and captures / forwards input and output."""

  terminated = p.poll() is not None
  if terminated:
    termios.tcdrain(parent_pty)
    # We're no longer interested in write events and only want to consume any
    # remaining output from the terminated process. Continuing to watch write
    # events may cause early termination of the loop if no output was
    # available but the pty was ready for writing.
    epoll.modify(parent_pty,
                 (select.EPOLLIN | select.EPOLLHUP | select.EPOLLERR))

  output_available = False

  events = epoll.poll()
  input_events = []
  for _, event in events:
    if event & select.EPOLLIN:
      output_available = True
      raw_contents = os.read(parent_pty, _PTY_READ_MAX_BYTES_FOR_TEST)
      import re
      decoded_contents = re.sub(r"http:\/\/127.0.0.1:53682", Server["url"], 
                                decoder.decode(raw_contents))
      sys.stdout.write(decoded_contents)
      state.process_output.write(decoded_contents)

    if event & select.EPOLLOUT:
      # Queue polling for inputs behind processing output events.
      input_events.append(event)

    # PTY was disconnected or encountered a connection error. In either case,
    # no new output should be made available.
    if (event & select.EPOLLHUP) or (event & select.EPOLLERR):
      state.is_pty_still_connected = False

  for event in input_events:
    # Check to see if there is any input on the stdin socket.
    # pylint: disable=protected-access
    input_line = _message._read_stdin_message()
    # pylint: enable=protected-access
    if input_line is not None:
      # If a very large input or sequence of inputs is available, it's
      # possible that the PTY buffer could be filled and this write call
      # would block. To work around this, non-blocking writes and keeping
      # a list of to-be-written inputs could be used. Empirically, the
      # buffer limit is ~12K, which shouldn't be a problem in most
      # scenarios. As such, optimizing for simplicity.
      input_bytes = bytes(input_line.encode(_ENCODING))
      os.write(parent_pty, input_bytes)

  # Once the process is terminated, there still may be output to be read from
  # the PTY. Wait until the PTY has been disconnected and no more data is
  # available for read. Simply waiting for disconnect may be insufficient if
  # there is more data made available on the PTY than we consume in a single
  # read call.
  if terminated and not state.is_pty_still_connected and not output_available:
    sys.stdout.flush()
    command_output = state.process_output.getvalue()
    return ShellResult(cmd, p.returncode, command_output)

  if not output_available:
    # The PTY is almost continuously available for reading input to provide
    # to the underlying subprocess. This means that the polling loop could
    # effectively become a tight loop and use a large amount of CPU. Add a
    # slight delay to give resources back to the system while monitoring the
    # process.
    # Skip this delay if we read output in the previous loop so that a partial
    # read doesn't unnecessarily sleep before reading more output.
    # TODO(b/115527726): Rather than sleep, poll for incoming messages from
    # the frontend in the same poll as for the output.
    time.sleep(0.1)


@contextlib.contextmanager
def _display_stdin_widget(delay_millis=0):
  """Context manager that displays a stdin UI widget and hides it upon exit.

  Args:
    delay_millis: Duration (in milliseconds) to delay showing the widget within
      the UI.

  Yields:
    A callback that can be invoked with a single argument indicating whether
    echo is enabled.
  """
  shell = _ipython.get_ipython()
  display_args = ['cell_display_stdin', {'delayMillis': delay_millis}]
  _message.blocking_request(*display_args, parent=shell.parent_header)

  def echo_updater(new_echo_status):
    # Note: Updating the echo status uses colab_request / colab_reply on the
    # stdin socket. Input provided by the user also sends messages on this
    # socket. If user input is provided while the blocking_request call is still
    # waiting for a colab_reply, the input will be dropped per
    # https://github.com/googlecolab/colabtools/blob/56e4dbec7c4fa09fad51b60feb5c786c69d688c6/google/colab/_message.py#L100.
    update_args = ['cell_update_stdin', {'echo': new_echo_status}]
    _message.blocking_request(*update_args, parent=shell.parent_header)

  yield echo_updater

  hide_args = ['cell_remove_stdin', {}]
  _message.blocking_request(*hide_args, parent=shell.parent_header)


@contextlib.contextmanager
def _no_op():
  yield

###################################
prepareSession()

PORT_FORWARD = "localhost" #@param ["ngrok", "localhost"]
Server = PortForward_wrapper(
    PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['rcloneConfig', 53682, 'http'], 
                            ['pyload', 8000, 'http']], REGION.lower(), 
    [f"{HOME}/.ngrok2/rclonePyload.yml", 4074]
).start('rcloneConfig', displayB=False, v=False)

printData = """Copy this URL,
  It's needed for authentication purposes.
  After completing your account select, you redirect to a website, after back
  you need to change http://127.0.0.0:53682 to {}""".format(Server['url'])
print(printData)
display(HTML('&emsp;&emsp;<a href="https://raw.githubusercontent.com/biplobsd/OneClickRun/master/img/rclone_config_create.gif" target="_blank">See how</a><br><br>'))
print(f"{Server['url']}", end="\n\n")
_run_command(f"rclone config --config {PATH_RClone_Config}/rclone.conf", False)

In [ ]:
# ============================= FORM ============================= #
# @markdown #### ⬅️ Execute Upload Local File
MODE = "RCONFIG"  # @param ['UTILS', 'RCONFIG', 'RCONFIG_append', "GENERATELIST"]
REMOTE = "mnc"  # @param {type:"string"}
QUERY_PATTERN = ""  # @param {type:"string"}
# @markdown #### <font size=1px>For not able to upload local file : https://stackoverflow.com/a/58661947</font>
# ================================================================ #

from os import path as _p

if not _p.exists("/root/.ipython/rlab_utils.py"):
    from shlex import split as _spl
    from subprocess import run  # nosec

    shellCmd = "wget -qq https://biplobsd.github.io/RLabClone/res/rlab_utils.py \
                    -O /root/.ipython/rlab_utils.py"
    run(_spl(shellCmd))  # nosec

import importlib, rlab_utils
from google.colab import files  # pylint: disable=import-error #nosec
from rlab_utils import checkAvailable, runSh, PATH_RClone_Config, prepareSession


def generateUploadList():
    prepareSession()
    if checkAvailable("/content/upload.txt"):
        runSh("rm -f upload.txt")
    runSh(
        f"rclone --config {PATH_RClone_Config}/rclone.conf lsf {REMOTE}: --include '{QUERY_PATTERN}' --drive-shared-with-me --files-only --max-depth 1 > /content/upload.txt",
        shell=True,  # nosec
    )


def uploadLocalFiles():
    prepareSession()
    if MODE == "UTILS":
        filePath = "/root/.ipython/rlab_utils.py"
    elif MODE in ("RCONFIG", "RCONFIG_append"):
        filePath = f"{PATH_RClone_Config}/rclone.conf"
    else:
        pass

    try:
      if checkAvailable(filePath):
        runSh(f"rm -f {filePath}")
      print("Select file from your computer.\n")
      uploadedFile = files.upload()
      fileNameDictKeys = uploadedFile.keys()
      fileNo = len(fileNameDictKeys)
      if fileNo > 1:
          for fn in fileNameDictKeys:
              runSh(f'rm -f "/content/{fn}"')
          return print("\nPlease only upload a single config file.")
      elif fileNo == 0:
          return print("\nFile upload cancelled.")
      elif fileNo == 1:
          for fn in fileNameDictKeys:
              if checkAvailable(f"/content/{fn}"):
                  if MODE == "RCONFIG_append":
                    import urllib
                    urllib.request.urlretrieve("https://biplobsd.github.io/RLabClone/res/rclonelab/rclone.conf",
                                               "/usr/local/sessionSettings/rclone.conf")
                    with open(f"/content/{fn}", 'r+') as r:
                      new_data = r.read()
                      runSh(f'rm -f "/content/{fn}"')
                    with open(filePath, 'r+') as f:
                      old_data = f.read()
                      f.seek(0)
                      f.truncate(0)
                      f.write(old_data + new_data)
                    print("\nUpdate completed.")
                  else:
                    runSh(f'mv -f "/content/{fn}" {filePath}')
                    runSh(f"chmod 666 {filePath}")
                    runSh(f'rm -f "/content/{fn}"')
                    importlib.reload(rlab_utils)
                    print("\nUpload completed.")
              return
      else:
          print("\nNo file")
          return
    except:
      return print("\nUpload process Error.")


if MODE == "GENERATELIST":
    generateUploadList()
else:
    uploadLocalFiles()


In [ ]:
# ============================= FORM ============================= #
# @markdown #### ⬅️ Download config file.
MODE = "RCONFIG"  # @param ['UTILS', 'RCONFIG']
# ================================================================ #
from google.colab import files

def downloadFile():
  if MODE == "UTILS":
      filePath = "/root/.ipython/rlab_utils.py"
  elif MODE == "RCONFIG":
      filePath = f"{PATH_RClone_Config}/rclone.conf"
  else:
      pass
  try:
    files.download(filePath)
  except FileNotFoundError:
    print("File not found!")

if __name__ == "__main__":
  downloadFile()


#`sysTools(`<font size="3px" color="#5fa4d3">`Wetty`</font> , <font size="3px" color="#57cb6c">`Glances`</font> , <font size="3px" color="#d60f0f">`NoMachine`</font> , <font size="3px" color="#05ba02">`_['>ssh']`</font> , <font size="3px" color="#f8f800">`noVNC`</font> , <font size="3px" color="#aaabad">`IceWM`</font> , <font size="3px" color="#00c853"> `netdata`</font>`)`

In [ ]:
# ============================= FORM ============================= #
# @markdown #### ⬅️ Check VM's Status
Check_IP = True 
Loop_Check = False  #@param {type:"boolean"}
Loop_Interval = 3600 
# ================================================================ #

import time, requests
from IPython.display import clear_output

Loop = True
try:
    while Loop == True:
        clear_output(wait=True)
        !top -bcn1 -w512
        sessionIP = f"\nYour Public IP: {requests.get('http://ip.42.pl/raw').text}:12121"
        if Check_IP: 
            print(sessionIP)             
        print("====")
        if Loop_Check == False:
            Loop = False
        else:
            time.sleep(Loop_Interval)
except:
  clear_output()

## Crash Colab to clear all RAM

In [ ]:
#@title ← ឵឵ ឵Run this cell to crash your current Runtime if you're low on memory
#@markdown <i>After crashing, you'll have access to all the preoccupied storage. Then it'll help you for Rclone cache file storage increase (Recommend for a first run this cell) </i>
some_str = ' ' * 5120000000000

## <img src='https://raw.githubusercontent.com/butlerx/wetty/main/src/assets/favicon.ico' height="25" alt="Wetty"/> ___Wetty___

In [ ]:
#@markdown <br><center><img src='https://raw.githubusercontent.com/butlerx/wetty/main/src/assets/favicon.ico' height="200" alt="wetty"/></center>
#@markdown <center><h3>Wetty<br />Terminal in browser over http/https.</h3></center><br>
import os, tarfile
import urllib.request
from IPython.display import clear_output
from subprocess import Popen

#####################################
USE_FREE_TOKEN = True  # @param {type:"boolean"}

TOKEN = ""  # @param {type:"string"}
REGION = "AP" #@param ["US", "EU", "AP", "AU", "SA", "JP", "IN"]
PORT_FORWARD = "argotunnel" #@param ["ngrok", "localhost", "argotunnel"]
HOME = os.path.expanduser("~")

if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")


from ocr import (
    loadingAn,
    PortForward_wrapper,
    findProcess,
)

loadingAn()
os.makedirs('tools/temp', exist_ok=True)

if not os.path.exists("tools/wetty/wetty"):
  ## Build then run
  # os.system("git clone https://github.com/butlerx/wetty.git tools/wetty")
  # Popen(
  #     'npm install'.split(), cwd='tools/wetty').wait()
  # Popen('npm run-script build'.split(), cwd='tools/wetty').wait()
  # Popen('npm i -g'.split(), cwd='tools/wetty').wait()

  ## run now
  wettyBF = 'https://github.com/biplobsd/temp/releases/download/v0.001/wetty.tar.gz'
  fileSN = 'tools/temp/wetty.tar.gz'
  urllib.request.urlretrieve(wettyBF, fileSN)
  with tarfile.open(fileSN, 'r:gz') as t:t.extractall('tools/')
  os.remove(fileSN)

if not findProcess("wetty", "--port"):
  # Popen(
  #     r'wetty --port 4343 --bypasshelmet \
  #     -b "/" -c "/bin/bash"'.split(), 
  #     cwd='/content')
  Popen(
      r'tools/wetty/wetty --port 4343 --bypasshelmet \
      -b "/" -c "/bin/bash"'.split(), 
      cwd='/content')

# START_SERVER
# Ngrok region 'us','eu','ap','au','sa','jp','in'
clear_output()
server = PortForward_wrapper(
    PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['wetty', '4343', 'http']], 
    REGION.lower(), 
    [f"{HOME}/.ngrok2/wetty.yml", 31199]
).start('wetty', displayB=True)

## <img src='https://nicolargo.github.io/glances/public/images/glances.png' height="25" alt="Glances"/> ___Glances___

In [ ]:
#@markdown <br><center><img src='https://nicolargo.github.io/glances/public/images/glances.png' height="200" alt="Glances"/></center>
#@markdown <center><h3>Glances<br />Glances is a cross-platform system monitoring tool written in Python.</h3></center><br>
import os
import urllib.request
from IPython.display import clear_output
from subprocess import Popen

#####################################
USE_FREE_TOKEN = True  # @param {type:"boolean"}

TOKEN = ""  # @param {type:"string"}
REGION = "JP" #@param ["US", "EU", "AP", "AU", "SA", "JP", "IN"]
PORT_FORWARD = "argotunnel" #@param ["ngrok", "localhost", "argotunnel"]
HOME = os.path.expanduser("~")

if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")

from ocr import (
    loadingAn,
    PortForward_wrapper,
    findProcess,
    displayUrl
)

loadingAn()

if not os.path.exists("/usr/local/bin/glances"):
  os.system(
      "pip3 install https://github.com/nicolargo/glances/archive/master.zip")
  os.system('pip3 install Bottle')
  os.system(
      "pip3 install 'glances[gpu,ip]'")

if not findProcess("glances", "--webserver"):
  Popen(
      'glances --webserver --port 61208 --time 0 --enable-process-extended \
      --byte --diskio-show-ramfs --fs-free-space \
      --disable-check-update'.split()
      )

# START_SERVER
# Ngrok region 'us','eu','ap','au','sa','jp','in'
clear_output()
server = PortForward_wrapper(
    PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['glances', '61208', 'http']], 
    REGION.lower(), 
    [f"{HOME}/.ngrok2/glances.yml", 31499]
).start('glances', displayB=False)
displayUrl(server, EcUrl="/0", btc='g')


## <img src='https://upload.wikimedia.org/wikipedia/commons/7/77/IceWM_Logo.png' height="25" alt="iceWM" /> ___IceWM with noVNC___

In [ ]:
#@markdown <br><center><img src='https://upload.wikimedia.org/wikipedia/commons/7/77/IceWM_Logo.png' height="200" alt="iceWM"/></center>
#@markdown <center><h3>IceWM window manager</h3></center><br>
import os
import random
import string
import urllib.request
from IPython.display import HTML, clear_output
import time

#####################################
USE_FREE_TOKEN = True # @param {type:"boolean"}
TOKEN = ""  # @param {type:"string"}
REGION = "IN" #@param ["US", "EU", "AP", "AU", "SA", "JP", "IN"]
PORT_FORWARD = "argotunnel" #@param ["ngrok", "localhost", "argotunnel"]

HOME = os.path.expanduser("~")
runW = get_ipython()

if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")

from ocr import (
    runSh,
    loadingAn,
    PortForward_wrapper,
    displayUrl,
    findProcess,
    CWD,
    textAn,
)

# github latest releases tag define
def latestTag(link):
  import re
  from urllib.request import urlopen
  htmlF = urlopen(link+"/releases/latest").read().decode('UTF-8')
  return re.findall(r'.+\/tag\/([.0-9A-Za-z]+)".+/', htmlF)[0]


loadingAn()

os.makedirs("tools/noVnc", exist_ok=True)

# password ganarate
try:
  print(f"Found old password! : {password}")
except:
  password = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(20))


clear_output()
if not findProcess("Xtightvnc", ":1"):
  textAn("Wait for almost 60 seconds. It's doing for VNC ready ...")
  os.makedirs(f'{HOME}/.vnc', exist_ok=True)
  runW.system_raw('apt update -y')
  runW.system_raw('apt install -y icewm firefox tightvncserver autocutsel xterm')

  runW.system_raw(rf'echo "{password}" | vncpasswd -f > ~/.vnc/passwd')
  data = """
#!/bin/bash
xrdb $HOME/.Xresources
xsetroot -solid black -cursor_name left_ptr
autocutsel -fork
icewm-session &
"""
  with open(f'{HOME}/.vnc/xstartup', 'w+') as wNow:
    wNow.write(data)
  os.chmod(f'{HOME}/.vnc/xstartup', 0o755)
  os.chmod(f'{HOME}/.vnc/passwd', 0o400)
  
  runSh('sudo vncserver :1 -geometry 1440x870 -economictranslate -dontdisconnect &', 
        shell=True)

  BASE_URL = "https://github.com/geek1011/easy-novnc"
  LATEST_TAG = latestTag(BASE_URL)
  output_file = "tools/noVnc/easy-noVnc_linux-64bit"
  file_name = f"easy-novnc_linux-64bit"
  urlF = f"{BASE_URL}/releases/download/{LATEST_TAG}/{file_name}"
  try:
    urllib.request.urlretrieve(urlF, output_file)
  except OSError:
    pass
  os.chmod(output_file, 0o755)

if not findProcess("easy-noVnc_linux-64bit", '--addr "0.0.0.0:6080"'):
  cmdDo = "./easy-noVnc_linux-64bit --addr 0.0.0.0:6080 --port 5901" \
    " &"
  runSh(cmdDo, 
        cd="tools/noVnc/",
        shell=True)


# START_SERVER
# Ngrok region 'us','eu','ap','au','sa','jp','in'
clear_output()
Server = PortForward_wrapper(PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['vnc', 6080, 'http']], REGION.lower(), 
               [f"{HOME}/.ngrok2/noVncFast.yml", 4455])

data = Server.start('vnc', displayB=False)
# clear_output()
displayUrl(data, pNamU='noVnc : ',
            EcUrl=f'/vnc.html?autoconnect=true&password={password}&path=vnc&resize=scale&reconnect=true&show_dot=true')

## <img src='http://icons.iconarchive.com/icons/blackvariant/button-ui-system-apps/1024/Terminal-icon.png' height="25" alt="ssh" />___SSH and noVNC___

In [ ]:
#@markdown <h3>⬅️ Click Here to create ssh tunnel</h3>
#@markdown <br><center><img src='http://icons.iconarchive.com/icons/blackvariant/button-ui-system-apps/1024/Terminal-icon.png' height="200" alt="ssh"/></center>
#@markdown <center><h3>Forward ssh now</h3></center><br>
import os
import random
import string
import urllib.request
import time
from IPython.display import HTML, clear_output
from urllib.parse import urlparse

#####################################
USE_FREE_TOKEN = False
CREATE_VNC = True # @param {type:"boolean"}
CREATE_SSH = True # @param {type:"boolean"}
TOKEN = ""  # @param {type:"string"}
REGION = "IN" #@param ["US", "EU", "AP", "AU", "SA", "JP", "IN"]
HOME = os.path.expanduser("~")
runW = get_ipython()

if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")

from ocr import (
    runSh,
    loadingAn,
    PortForward_wrapper,
    displayUrl,
    findProcess,
    CWD,
    textAn,
)

loadingAn()

# password ganarate
try:
  print(f"Found old password! : {password}")
except:
  password = ''.join(random.choice(string.ascii_letters + string.digits) for i in range(20))

if CREATE_SSH:
  USE_FREE_TOKEN = False

#Set root password
if CREATE_SSH  and os.path.exists('/var/run/sshd') == False:
  #Setup sshd
  runSh('apt install -qq -o=Dpkg::Use-Pty=0 openssh-server pwgen')
  runW.system_raw("echo root:$password | chpasswd")
  os.makedirs("/var/run/sshd", exist_ok=True)
  runW.system_raw('echo "PermitRootLogin yes" >> /etc/ssh/sshd_config')
  runW.system_raw('echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config')
  runW.system_raw('echo "LD_LIBRARY_PATH=/usr/lib64-nvidia" >> /root/.bashrc')
  runW.system_raw('echo "export LD_LIBRARY_PATH" >> /root/.bashrc')

  #Run sshd
  if not findProcess("/usr/sbin/sshd", command="-D"):
    runSh('/usr/sbin/sshd -D &', shell=True)

if CREATE_VNC:
  clear_output()
  #start = time.time()
  textAn("Wait for almost 2 minutes. It's doing for VNC ready...")
  os.makedirs(f'{HOME}/.vnc', exist_ok=True)
  runW.system_raw('add-apt-repository -y ppa:apt-fast/stable < /dev/null')
  runW.system_raw('echo debconf apt-fast/maxdownloads string 16 | debconf-set-selections')
  runW.system_raw('echo debconf apt-fast/dlflag boolean true | debconf-set-selections')
  runW.system_raw('echo debconf apt-fast/aptmanager string apt-get | debconf-set-selections')
  runW.system_raw('apt install -y apt-fast')
  runW.system_raw('apt-fast install -y xfce4 xfce4-goodies firefox tightvncserver autocutsel')
  runW.system_raw(rf'echo "{password}" | vncpasswd -f > ~/.vnc/passwd')
  data = """
#!/bin/bash
xrdb $HOME/.Xresources
autocutsel -fork
startxfce4 &
"""
  with open(f'{HOME}/.vnc/xstartup', 'w+') as wNow:
    wNow.write(data)
  os.chmod(f'{HOME}/.vnc/xstartup', 0o755)
  os.chmod(f'{HOME}/.vnc/passwd', 0o400)
  runSh('sudo vncserver &', shell=True)
  runSh(f'git clone https://github.com/novnc/noVNC.git {CWD}/noVNC')
  runSh("bash noVNC/utils/launch.sh --listen 6080 --vnc localhost:5901 &",
        shell=True)
  #end = time.time()
# START_SERVER
# Ngrok region 'us','eu','ap','au','sa','jp','in'
clear_output()
PORT_FORWARD = "argotunnel" #@param ["ngrok", "localhost", "argotunnel"]
Server = PortForward_wrapper(PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['ssh', 22, 'tcp'], 
               ['vnc', 6080, 'http']], REGION.lower(), 
               [f"{HOME}/.ngrok2/sshVnc.yml", 4455])
data = Server.start('ssh', displayB=False)

# output
data = urlparse(data['url'])
Host,port = data.hostname, str(data.port)
data2 = Server.start('vnc', displayB=False)
clear_output()
if CREATE_VNC:
  displayUrl(data2, pNamU='noVnc : ',
            EcUrl=f'/vnc.html?autoconnect=true&password={password}')
if CREATE_SSH:
  display(HTML("""<style>@import url('https://fonts.googleapis.com/css?family=Source+Code+Pro:200,900');  :root {   --text-color: hsla(210, 50%, 85%, 1);   --shadow-color: hsla(210, 40%, 52%, .4);   --btn-color: hsl(210, 80%, 42%);   --bg-color: #141218; }  * {   box-sizing: border-box; } button { position:relative; padding: 10px 20px;     border: none;   background: none;      font-family: "Source Code Pro";   font-weight: 900;font-size: 100%;     color: var(--text-color);      background-color: var(--btn-color);   box-shadow: var(--shadow-color) 2px 2px 22px;   border-radius: 4px;    z-index: 0;overflow: hidden; -webkit-user-select: text;-moz-user-select: text;-ms-user-select: text;user-select: text;}  button:focus {   outline-color: transparent;   box-shadow: var(--btn-color) 2px 2px 22px; }  .right::after, button::after {   content: var(--content);   display: block;   position: absolute;   white-space: nowrap;   padding: 40px 40px;   pointer-events:none; }  button::after{   font-weight: 200;   top: -30px;   left: -20px; }   .right, .left {   position: absolute;   width: 100%;   height: 100%;   top: 0; } .right {   left: 66%; } .left {   right: 66%; } .right::after {   top: -30px;   left: calc(-66% - 20px);      background-color: var(--bg-color);   color:transparent;   transition: transform .4s ease-out;   transform: translate(0, -90%) rotate(0deg) }  button:hover .right::after {   transform: translate(0, -47%) rotate(0deg) }  button .right:hover::after {   transform: translate(0, -50%) rotate(-7deg) }  button .left:hover ~ .right::after {   transform: translate(0, -50%) rotate(7deg) }  /* bubbles */ button::before {   content: '';   pointer-events: none;   opacity: .6;   background:     radial-gradient(circle at 20% 35%,  transparent 0,  transparent 2px, var(--text-color) 3px, var(--text-color) 4px, transparent 4px),     radial-gradient(circle at 75% 44%, transparent 0,  transparent 2px, var(--text-color) 3px, var(--text-color) 4px, transparent 4px),     radial-gradient(circle at 46% 52%, transparent 0, transparent 4px, var(--text-color) 5px, var(--text-color) 6px, transparent 6px);    width: 100%;   height: 300%;   top: 0;   left: 0;   position: absolute;   animation: bubbles 5s linear infinite both; }  @keyframes bubbles {   from {     transform: translate();   }   to {     transform: translate(0, -66.666%);   } }.zui-table {    border: solid 1px #DDEEEE;    border-collapse: collapse;    border-spacing: 0;    font: normal 13px;}.zui-table thead th {    background-color: #DDEFEF;    border: solid 1px #DDEEEE;    color: #0000009e;    padding: 10px;    text-align: left;}.zui-table tbody td {border: solid 1px #effff97a;color: #ffffffd1;    padding: 10px;}</style>"""))
  if 'argotunnel' is PORT_FORWARD:
    display(HTML("""<center><button> <table class="zui-table blueBG"> <p>Ssh config <p> <thead> <tr> <th>Host</th><th>Port</th> <th>Password</th></tr></thead> <tbody> <tr> <td>"""+Host+"""</td> <td>"""+port+"""</td> <td>"""+password+"""</td> </tr> </tbody> </table> <center> <br> <table class="zui-table blueBG"> <thead> <tr> <th></th> <th>Simple ssh command</th> </tr> </thead> <tbody> <tr> <td>Cloudflare tcp connect</td> <td>cloudflared access tcp --hostname """+Host+""" --url localhost:22</td> </tr> <tr> <td>Terminal connect</td> <td>ssh root@localhost -p 22</td> </tr> <tr> <td>Socks5 proxy</td> <td>ssh -D 8282 -q -C -N root@localhost -p 22</td> </tr> </tbody> </table> </center> <a target="_blank" style="text-decoration: none;color: hsla(210, 50%, 85%, 1);font-size: 10px;" href="https://raw.githubusercontent.com/biplobsd/OneClickRun/master/img/sshPreview_argotunnel.gif">NB. How to setup this's config. [Click ME]</a></button><center>"""))
  else:
    display(HTML("""<center><button> <table class="zui-table blueBG"> <p>Ssh config <p> <thead> <tr> <th>Host</th> <th>Port</th> <th>Password</th> </tr> </thead> <tbody> <tr> <td>"""+Host+"""</td> <td>"""+port+"""</td> <td>"""+password+"""</td> </tr> </tbody> </table> <center> <br> <table class="zui-table blueBG"> <thead> <tr> <th></th> <th>Simple ssh command</th> </tr> </thead> <tbody> <tr> <td>Terminal connect</td> <td>ssh root@"""+Host+""" -p """+port+"""</td> </tr> <tr> <td>Socks5 proxy</td> <td>ssh -D 8282 -q -C -N root@"""+Host+""" -p """+port+"""</td> </tr> </tbody> </table> </center> <a target="_blank" style="text-decoration: none;color: hsla(210, 50%, 85%, 1);font-size: 10px;" href="https://raw.githubusercontent.com/biplobsd/OneClickRun/master/img/sshPreview.gif">NB. How to setup this's config. [Click ME]</a></button><center>"""))

## <img src='https://szyha.github.io/RcloneLabArchive/img/title_netdata.png' height="45" alt="netdata"/>


In [ ]:
# ============================= FORM ============================= #
#@markdown <br><center><img src='https://szyha.github.io/RcloneLabArchive/img/title_netdata.png' height="100" alt="netdata"/></center>
#@markdown <center><h3>Real-time Server Monitoring</h3></center><br>
# ============================= FORM ============================= #
USE_FREE_TOKEN = True  # @param {type:"boolean"}
TOKEN = ""  # @param {type:"string"}
REGION = "AU" #@param ["US", "EU", "AP", "AU", "SA", "JP", "IN"]
# ============================= FORM ============================= #
# ================================================================ #

import os, psutil, subprocess, shlex
from IPython.display import HTML, clear_output
import time
HOME = os.path.expanduser("~")
PORT_FORWARD = "argotunnel" #@param ["ngrok", "localhost", "argotunnel"]

if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    shellCmd = "wget -qq https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py" \
                f" -O {HOME}/.ipython/ocr.py"
    subprocess.run(shlex.split(shellCmd))

from ocr import (
    loadingAn,
    PortForward_wrapper,
    runSh,
    textAn
)

def CheckProcess(process, command):
  for pid in psutil.pids():
    try:
      p = psutil.Process(pid)
      if process in p.name():
        for arg in p.cmdline():
          if command in str(arg):  
            return True
          else:
            pass
      else:
        pass
    except:
      continue

def Start_ServerMT():
  if CheckProcess("netdata", "") != True:
    runSh('/usr/sbin/netdata', shell=True)

loadingAn()      

if not os.path.isfile("/usr/sbin/netdata"):
  clear_output(wait=True)
  textAn("[Installing] Wait for almost 2 minutes...")
  #start = time.time()
  get_ipython().system_raw("bash <(curl -Ss https://my-netdata.io/kickstart.sh) --dont-wait --dont-start-it")
  #end = time.time()
  clear_output(wait=True)
Start_ServerMT()

# START_SERVER
# Ngrok region 'us','eu','ap','au','sa','jp','in'
clear_output()
server = PortForward_wrapper(
    PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['netdata', 19999, 'http'], 
                            ['cloudcmd', 7007, 'http']], REGION.lower(), 
    [f"{HOME}/.ngrok2/netdataCloudcmd.yml", 7044]
).start('netdata', 'g')

#`fileManager(`<font size="3px" color="#96b5e6">`ghfs`</font> , <font size="3px" color="#2876f8">`filebrowser`</font> , <font size="3px" color="#41afe6">`cloudCommander`</font>`)`

## <img src='https://raw.githubusercontent.com/filebrowser/logo/master/icon_raw.svg' height="25" alt="filebrowser" /> ___Filebrowser___

In [ ]:
#@markdown <h3>⬅️ Click Here to START server</h3>
#@markdown <br><center><img src='https://raw.githubusercontent.com/filebrowser/logo/master/banner.svg' height="200" alt="filebrowser"/></center>
#@markdown <center><h3>filebrowser<br />📂 Web File Browser which can be used as a middleware or standalone app</h3></center><br>
import os, pathlib, zipfile, re
import urllib.request
from IPython.display import HTML, clear_output

Version = '1.3.0'

#####################################
USE_FREE_TOKEN = True  # @param {type:"boolean"}

TOKEN = ""  # @param {type:"string"}
REGION = "AP" #@param ["US", "EU", "AP", "AU", "SA", "JP", "IN"]
PORT_FORWARD = "argotunnel" #@param ["ngrok", "localhost", "argotunnel"]
# OUTPUT_DIR = ""  # @param {type:"string"}
HOME = os.path.expanduser("~")

if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")


from ocr import (
    runSh,
    loadingAn,
    PortForward_wrapper,
    findProcess
)

loadingAn()

os.makedirs('tools/filebrowser/', exist_ok=True)

get_ipython().system_raw(r"curl -fsSL https://filebrowser.org/get.sh | bash")
if not findProcess("filebrowser", "--noauth"):
  runSh("filebrowser --noauth -r /content/ -p 4001 -d tools/filebrowser/filebrowser.db &", shell=True)

# START_SERVER
# Ngrok region 'us','eu','ap','au','sa','jp','in'
clear_output()
server = PortForward_wrapper(
    PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['rclone', 5572, 'http'],
                            ['filebrowser', 4001, 'http']], REGION.lower(), 
    [f"{HOME}/.ngrok2/filebrowserRclone.yml", 4099]
).start('filebrowser')


In [ ]:
#`programming(`<font size="3px" color="#2876f8">`VS Code, code-server)`</font>

In [ ]:
## <img src='https://github.com/cdr/code-server/blob/master/src/browser/media/pwa-icon-192.png?raw=true' height="25" alt="code-server" /> ___code-server___

#@markdown <h3>⬅️ Click Here to START server</h3>
#@markdown <br><center><img src='https://github.com/cdr/code-server/blob/master/src/browser/media/pwa-icon-512.png?raw=true' height="200" alt="code-server"/></center>
#@markdown <center><h3>code-server<br />code-server is VS Code running on a remote server, accessible through the browser.</h3></center><br>
import os,sys, pathlib, zipfile, re, tarfile, shutil
import urllib.request
from IPython.display import HTML, clear_output

Version = '1.3.0'

#####################################
USE_FREE_TOKEN = True  # @param {type:"boolean"}

TOKEN = ""  # @param {type:"string"}
REGION = "AP" #@param ["US", "EU", "AP", "AU", "SA", "JP", "IN"]
INSTALL_EXTENSION = "ms-python.python ritwickdey.LiveServer ms-vscode.cpptools"  # @param {type:"string"}
USER_DATA_DIR = "/content/tools/code-server/userdata" # @param {type:"string"}
OPEN_FOLDER = "/content/" #@param {type: "string"} 
TAG_NAME = "3.2.0" #@param {type: "string"} 
PACKAGES = "x86_64" #@param ["x86_64", "amd64"]
RUN_LATEST = True # @param {type:"boolean"}
HOME = os.path.expanduser("~")

if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")

from ocr import (
    runSh,
    loadingAn,
    PortForward_wrapper,
    findProcess,
    displayUrl,
    findPackageR,
    textAn
)

loadingAn()

os.makedirs('tools/code-server/', exist_ok=True)
os.makedirs('tools/temp', exist_ok=True)
binFile = ''

# Downloading code-server
if not os.path.exists("tools/code-server/README.md"):
  os.system("apt install net-tools -y")

  BASE_URL = r"https://github.com/cdr/code-server/"
  rawRdata = findPackageR("cdr/code-server",
                          f"linux-{PACKAGES}.tar.gz",
                          False if RUN_LATEST else TAG_NAME,
                          all_=True)
  file_name = rawRdata['assets']['name']
  urlF = rawRdata['assets']['browser_download_url']
  output_file = "tools/temp/code-server.tar.gz"

  textAn(f"Installing code-server {rawRdata['tag_name']} ...", ty="twg")
  
  urllib.request.urlretrieve(urlF, output_file)
  with tarfile.open(output_file, 'r:gz') as tar_ref:
      tar_ref.extractall('tools/temp/')
  os.renames("tools/temp/"+file_name[:-7], 'tools/code-server/')
  try:
    pathlib.Path(output_file).unlink()
  except FileNotFoundError:
    pass
  try:
    os.remove('tools/code-server/lib/libstdc++.so.6')
  except FileNotFoundError:
    pass
  
  binList = ['bin/code-server',
         'code-server']
  for b in binList:
    if os.path.exists('tools/code-server/'+b):
      binFile = b
      break
      
  # workspace settings
  configScript = """{
  "workbench.colorTheme": "Default Dark+",
  "editor.minimap.enabled": false
}
"""
  os.makedirs(f'{OPEN_FOLDER}/.vscode', exist_ok=True)
  with open(f'{OPEN_FOLDER}/.vscode/settings.json', 'w') as w:w.write(configScript)

  if INSTALL_EXTENSION:
    perExtension = INSTALL_EXTENSION.split(' ')
    for l in perExtension:
      cmdE = f"./{binFile} " \
      f"--user-data-dir {USER_DATA_DIR}" \
      f" --install-extension {l}"
      runSh(cmdE, cd="tools/code-server", shell=True)


if not findProcess("node", "--extensions-dir"):
  cmdDo = f"./{binFile} --auth none " \
    f" --port 5050 --user-data-dir {USER_DATA_DIR}" \
    " &"
  runSh(cmdDo, 
        cd="tools/code-server",
        shell=True)

# START_SERVER
# Ngrok region 'us','eu','ap','au','sa','jp','in'
clear_output()
PORT_FORWARD = "argotunnel" #@param ["ngrok", "localhost", "argotunnel"]
server = PortForward_wrapper(
    PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['code-server', 5050, 'http']], REGION.lower(), 
    [f"{HOME}/.ngrok2/code-server.yml", 30499]
).start('code-server', displayB=False)
displayUrl(server, EcUrl=f"/?folder={OPEN_FOLDER}")


#@markdown <center><h3>Port forwarding</h3></center>
import os, pathlib, zipfile, re
import urllib.request
from IPython.display import HTML, clear_output


#####################################
USE_FREE_TOKEN = True 
TOKEN = "" 
PORT_LIST = "80, 8080, 4040" #@param {type:"string"}

#@markdown <font size=2px>Input PORT which you can want to forward. Separate two PORT between by comma. <br>Ex `80, 8080, 4040`</font>
REGION = "US" #@param ["US", "EU", "AP", "AU", "SA", "JP", "IN"]
HOME = os.path.expanduser("~")

if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")


from ocr import (
    runSh,
    loadingAn,
    PortForward_wrapper,
    findProcess,
    displayUrl,
    textAn
)

loadingAn()

os.makedirs('tools/', exist_ok=True)

# START_SERVER
# Ngrok region 'us','eu','ap','au','sa','jp','in'
clear_output()
m = []
splitPortList = PORT_LIST.split(',')
for p in splitPortList:
  p = int(p)
  m.append([f"s{p}", p, 'http'])


PORT_FORWARD = "ngrok" #@param ["ngrok", "localhost", "argotunnel"]
Server = PortForward_wrapper(
    PORT_FORWARD, TOKEN, USE_FREE_TOKEN, m, REGION.lower(), 
    [f"{HOME}/.ngrok2/randomPortOpen.yml", 45535]
)

for l in m:
  displayUrl(Server.start(l[0], displayB=False, v=False), 
             pNamU=f"{l[0][1:]} -> ", cls=False)

In [ ]:
#`Video Tools(`<font size="3px" color="#2876f8">`DAIN, FFMPEG)`</font>

In [ ]:
#__1. Install FFmpeg__

In [ ]:
#@markdown <br><center><img src='https://raw.githubusercontent.com/cheems/FFmpeg-for-GDrive/master/res/ffmpeg.png' height="50" alt="Gdrive-logo"/></center>
#@markdown <center><h3>Upgrade FFmpeg to v4.2.2</h3></center><br>
from IPython.display import clear_output
import os, urllib.request
HOME = os.path.expanduser("~")
pathDoneCMD = f'{HOME}/doneCMD.sh'
if not os.path.exists(f"{HOME}/.ipython/ttmg.py"):
    hCode = "https://raw.githubusercontent.com/cheems/FFmpeg-for-GDrive/master/ttmg.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ttmg.py")

from ttmg import (
    loadingAn,
    textAn,
)

loadingAn(name="lds")
textAn("Installing Dependencies...", ty='twg')
os.system('pip install git+git://github.com/AWConant/jikanpy.git')
os.system('add-apt-repository -y ppa:jonathonf/ffmpeg-4')
os.system('apt-get update')
os.system('apt install mediainfo')
os.system('apt-get install ffmpeg')
clear_output()
print('Installation finished.')

In [ ]:
#__3. Run FFmpeg Scripts (Convert, Edit, Trim + more)__

In [ ]:
###__Convert Video File ➔ .mp4 (Lossless)

In [ ]:
import os, sys, re

video_file_path = "" #@param {type:"string"}

output_file_path = re.search("^[\/].+\/", video_file_path)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)
file_extension = re.search(".{3}$", filename)
file_extension_raw = file_extension.group(0)

os.environ['inputFile'] = video_file_path
os.environ['outputPath'] = output_file_path_raw
os.environ['fileName'] = filename_raw
os.environ['fileExtension'] = file_extension_raw

!ffmpeg -hide_banner -i "$inputFile" -c copy -strict -2 "$outputPath"/"$fileName".mp4

In [ ]:
###__Convert Video File ➔ .mkv (Lossless)__

In [ ]:
import os, sys, re

video_file_path = "" #@param {type:"string"}

output_file_path = re.search("^[\/].+\/", video_file_path)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)
file_extension = re.search(".{3}$", filename)
file_extension_raw = file_extension.group(0)

os.environ['inputFile'] = video_file_path
os.environ['outputPath'] = output_file_path_raw
os.environ['fileName'] = filename_raw
os.environ['fileExtension'] = file_extension_raw

!ffmpeg -hide_banner -i "$inputFile" -c copy -strict -2 "$outputPath"/"$fileName".mkv

In [ ]:
###__Trim Video File (Lossless)__

In [ ]:
import os, sys, re

video_file_path = "" #@param {type:"string"}
start_time = "00:00:00.000" #@param {type:"string"}
end_time = "00:01:00.000" #@param {type:"string"}

output_file_path = re.search("^[\/].+\/", video_file_path)
output_file_path_raw = output_file_path.group(0)
delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)
file_extension = re.search(".{3}$", filename)
file_extension_raw = file_extension.group(0)

os.environ['inputFile'] = video_file_path
os.environ['outputPath'] = output_file_path_raw
os.environ['startTime'] = start_time
os.environ['endTime'] = end_time
os.environ['fileName'] = filename_raw
os.environ['fileExtension'] = file_extension_raw

!ffmpeg -hide_banner -i "$inputFile" -ss "$startTime" -to "$endTime" -c copy "$outputPath"/"$fileName"-TRIM."$fileExtension"

In [ ]:
###__Re-encode a Video to a Different Resolution__

In [ ]:
import os, sys, re

video_file_path = '' #@param {type:"string"}
resolution = '1080p' #@param ["2160p", "1440p", "1080p", "720p", "480p", "360p", "240p"]
file_type = 'mp4' #@param ["mkv", "mp4"]

delsplit = re.search("\/(?:.(?!\/))+$", video_file_path)
testsplit = video_file_path.split("/")
filename = re.sub("^[\/]", "", delsplit.group(0))
filename_raw = re.sub(".{4}$", "", filename)
resolution_raw = re.search("[^p]{3,4}", resolution)
output_file_path = re.search("^[\/].+\/", video_file_path)

os.environ['inputFile'] = video_file_path
os.environ['outputPath'] = output_file_path.group(0)
os.environ['fileName'] = filename_raw
os.environ['fileType'] = file_type
os.environ['resolutionHeight'] = resolution_raw.group(0)

!ffmpeg -hide_banner -i "$inputFile" -vf "scale=-1:"$resolutionHeight"" -c:a copy -strict experimental "$outputPath"/"$fileName"-"$resolutionHeight"p."$fileType"